Install the necessary libraries and import the necessary packages

In [1]:
!apt-get install p7zip-full

!pip install SimpleITK
!pip install 'diskcache==5.2.1'
!pip install 'cassandra-driver==3.25.0'
import SimpleITK as sitk
import numpy as np

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 47.4MB 89kB/s 
     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 3.8MB 5.1MB/s 


In [2]:
!rm -rf lungCancerSegmentation/
!git clone https://github.com/jeffreyboschman/lungCancerSegmentation.git

Cloning into 'lungCancerSegmentation'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 92 (delta 45), reused 58 (delta 18), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [3]:
from lungCancerSegmentation.code.datasets import getCandidateInfoList, getCt, LunaDataset


In [4]:
#data from: https://luna16.grand-challenge.org/Download/

In [9]:
!wget -c -O zipped_subset_0.zip https://zenodo.org/record/3723295/files/subset0.zip?download=1

--2021-06-08 00:16:35--  https://zenodo.org/record/3723295/files/subset0.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 503 Service Temporarily Unavailable
2021-06-08 00:16:36 ERROR 503: Service Temporarily Unavailable.



In [6]:
!mkdir /content/data
!7z e zipped_subset_0.zip -o/content/data/subset_0 #unzips files 



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 0 bytes

Extracting archive: zipped_subset_0.zip
ERROR: zipped_subset_0.zip
Can not open the file as archive

  0%    
Can't open as archive: 1
Files: 0
Size:       0
Compressed: 0


Exploring the data

In [7]:

mhd_path = "/content/data/subset_0/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.mhd"
ct_mhd = sitk.ReadImage(mhd_path)
print(ct_mhd.GetOrigin()) #the offset of the origin in mm 
print(ct_mhd.GetSpacing()) #size of each voxel in mm
print(ct_mhd.GetDirection()) #flattened transformation matrix

RuntimeError: ignored

In [15]:
import numpy as np
x = np.array(ct_mhd.GetOrigin())
x.shape

NameError: ignored

In [10]:
candidateInfo_list = getCandidateInfoList(requireOnDisk_bool=False)
positiveInfo_list = [x for x in candidateInfo_list if x[0]]
diameter_list = [x[1] for x in positiveInfo_list]

In [11]:
print(len(positiveInfo_list))
print(positiveInfo_list[0])

1351
CandidateInfoTuple(isNodule_bool=True, diameter_mm=32.27003025, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886', center_xyz=(67.61451718, 85.02525992, -109.8084416))


In [12]:
for i in range(0, len(diameter_list), 100):
    print('{:4}  {:4.1f} mm'.format(i, diameter_list[i]))

   0  32.3 mm
 100  17.7 mm
 200  13.0 mm
 300  10.0 mm
 400   8.2 mm
 500   7.0 mm
 600   6.3 mm
 700   5.7 mm
 800   5.1 mm
 900   4.7 mm
1000   4.0 mm
1100   0.0 mm
1200   0.0 mm
1300   0.0 mm


In [13]:
for candidateInfo_tup in positiveInfo_list[:10]:
    print(candidateInfo_tup)
for candidateInfo_tup in positiveInfo_list[-10:]:
    print(candidateInfo_tup)

CandidateInfoTuple(isNodule_bool=True, diameter_mm=32.27003025, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886', center_xyz=(67.61451718, 85.02525992, -109.8084416))
CandidateInfoTuple(isNodule_bool=True, diameter_mm=30.61040636, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800', center_xyz=(47.90350511, 37.60442008, -99.93417567))
CandidateInfoTuple(isNodule_bool=True, diameter_mm=30.61040636, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800', center_xyz=(44.19, 37.79, -107.01))
CandidateInfoTuple(isNodule_bool=True, diameter_mm=30.61040636, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800', center_xyz=(40.69, 32.19, -97.15))
CandidateInfoTuple(isNodule_bool=True, diameter_mm=27.44242293, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.943403138251347598519939390311', center_xyz=(-45.29440163, 74.86925386, -97.52812481))
CandidateInfoTuple(isNodule_bool=True, diameter_mm=27.

In [16]:
np.histogram(diameter_list)


(array([323, 466, 248, 111,  71,  57,  37,  29,   5,   4]),
 array([ 0.        ,  3.22700302,  6.45400605,  9.68100907, 12.9080121 ,
        16.13501512, 19.36201815, 22.58902117, 25.8160242 , 29.04302722,
        32.27003025]))